In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [ ]:
from keras.callbacks import EarlyStopping, TensorBoard

In [ ]:
from google.colab import files
files.upload()

In [ ]:
import pickle
from collections import Counter

In [ ]:
def find_max_list(list):
    list_len = [len(i) for i in list]
    print(max(list_len))

In [ ]:
df = pd.read_csv('/content/final_data_training_done.csv')
df.drop('Unnamed: 0',axis=1)
#df.head()

In [ ]:
results = Counter() 
hitung_sp_char = df['Tweet'].str.lower().str.split().apply(results.update) 
print(len(results)) 

29170


In [ ]:
max_features = 29200
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(df['Tweet'].values)
X = tokenizer.texts_to_sequences(df['Tweet'].values)

In [ ]:
with open('token.pickle','wb') as handle:
  pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)
  print('tokenizer.pickle has been created')

tokenizer.pickle has been created


In [ ]:
find_max_list(X)

95


In [ ]:
XX = pad_sequences(X, maxlen=128) #padding

In [ ]:
Y = pd.get_dummies(df['Sentimen']).values

In [ ]:
print(Y)

[[0 0 1]
 [0 1 0]
 [0 0 1]
 ...
 [1 0 0]
 [0 1 0]
 [0 1 0]]


In [ ]:
X_Train, X_Test, y_train, y_test = train_test_split(XX,Y,test_size=0.2,random_state=42)

In [ ]:
embed_dim = 128
lstm_out = 196

"""model = Sequential()
model.add(Embedding(max_features, embed_dim))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2, input_length=XX.shape[1]))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())"""

units = 196

model = Sequential()
model.add(Embedding(max_features,embed_dim, input_length=XX.shape[1]))
model.add(LSTM(units,dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 128, 128)          3737600   
                                                                 
 lstm_4 (LSTM)               (None, 196)               254800    
                                                                 
 dense_4 (Dense)             (None, 3)                 591       
                                                                 
Total params: 3,992,991
Trainable params: 3,992,991
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
batch_size = 32
es = EarlyStopping(monitor='var_loss', mode='min', verbose=1)
history = model.fit(X_Train,y_train, epochs=10, batch_size=10, validation_data=(X_Test,y_test), verbose=1, callbacks=[es])

Epoch 1/10
1497/1497 [==============================] - ETA: 0s - loss: 0.5682 - accuracy: 0.7488

1497/1497 [==============================] - 955s 636ms/step - loss: 0.5682 - accuracy: 0.7488 - val_loss: 0.4551 - val_accuracy: 0.8057
Epoch 2/10
1497/1497 [==============================] - ETA: 0s - loss: 0.2899 - accuracy: 0.8871

1497/1497 [==============================] - 941s 629ms/step - loss: 0.2899 - accuracy: 0.8871 - val_loss: 0.4789 - val_accuracy: 0.8017
Epoch 3/10
1497/1497 [==============================] - ETA: 0s - loss: 0.1550 - accuracy: 0.9445

1497/1497 [==============================] - 944s 631ms/step - loss: 0.1550 - accuracy: 0.9445 - val_loss: 0.5301 - val_accuracy: 0.7996
Epoch 4/10
1497/1497 [==============================] - ETA: 0s - loss: 0.0846 - accuracy: 0.9708

1497/1497 [==============================] - 948s 633ms/step - loss: 0.0846 - accuracy: 0.9708 - val_loss: 0.7872 - val_accuracy: 0.7873
Epoch 5/10
1497/1497 [==============================] - ETA: 0s - loss: 0.0520 - accuracy: 0.9818

...

1497/1497 [==============================] - 943s 630ms/step - loss: 0.0328 - accuracy: 0.9876 - val_loss: 0.8482 - val_accuracy: 0.7886
Epoch 7/10
1497/1497 [==============================] - ETA: 0s - loss: 0.0285 - accuracy: 0.9894

1497/1497 [==============================] - 949s 634ms/step - loss: 0.0285 - accuracy: 0.9894 - val_loss: 0.9854 - val_accuracy: 0.7913
Epoch 8/10
1497/1497 [==============================] - ETA: 0s - loss: 0.0203 - accuracy: 0.9923

1497/1497 [==============================] - 946s 632ms/step - loss: 0.0203 - accuracy: 0.9923 - val_loss: 0.9568 - val_accuracy: 0.7801
Epoch 9/10
1497/1497 [==============================] - ETA: 0s - loss: 0.0157 - accuracy: 0.9935

1497/1497 [==============================] - 937s 626ms/step - loss: 0.0157 - accuracy: 0.9935 - val_loss: 1.0394 - val_accuracy: 0.7929
Epoch 10/10
1497/1497 [==============================] - ETA: 0s - loss: 0.0140 - accuracy: 0.9940

1497/1497 [==============================] - 941s 628ms/step - loss: 0.0140 - accuracy: 0.9940 - val_loss: 1.1011 - val_accuracy: 0.7934


In [ ]:
model.save('lstm_sentiment_model.h5',save_format='h5')


AttributeError: ignored

In [ ]:
score = model.evaluate(X_Train,y_train, verbose=1)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

468/468 [==============================] - 48s 103ms/step - loss: 0.0056 - accuracy: 0.9970
accuracy: 99.70%
